In [1]:
import json

from pathlib import Path

from tqdm.notebook import tqdm

import cv2 as cv
import numpy as np
import pandas as pd

In [2]:
base_dir = Path("/workspace/hrrsxb/AIP/Morphometrics/workflows/")

In [3]:
with open(base_dir / "videos.txt") as f:
    video_paths = f.readlines()
    video_paths = [Path(p.strip()) for p in video_paths]
    print(f"{len(video_paths)} videos")

48 videos


In [4]:
video_qualities = {
    'GX020200': ['clear', 'bright', 'gopro'],
    'GX010200': ['clear', 'bright', 'gopro'],
    'GX020199': ['clear', 'bright', 'gopro'],
    'GX010199': ['clear', 'bright', 'gopro'],
    'F1_20221116': ['murky', 'dim', 'phone'],
    'F4_20221116': ['murky', 'dim', 'phone'],
    'GX012191': ['murky', 'bright', 'gopro'],
    'GX012189': ['murky', 'bright', 'gopro'],
    'GX012190': ['murky', 'bright', 'gopro'],
    'GX012188': ['murky', 'bright', 'gopro'],
    'GX020006': ['murky', 'dim', 'gopro'],
    'GX010006': ['murky', 'dim', 'gopro'],
    'GX010033': ['clear', 'bright', 'gopro'],
    'GX010032': ['clear', 'bright', 'gopro'],
    'GX040026': ['clear', 'bright', 'gopro'],
    'GX030026': ['clear', 'bright', 'gopro'],
    'GX020026': ['clear', 'bright', 'gopro'],
    'GX010027': ['NA', 'NA', 'gopro'],
    'GX010026': ['clear', 'bright', 'gopro'],
    'GX030022': ['clear', 'bright', 'gopro'],
    'GX020022': ['clear', 'bright', 'gopro'],
    'GX010022': ['clear', 'bright', 'gopro'],
    '2022-03-15_11-46-18_B5': ['clear', 'bright', 'gopro'],
    'GH050197': ['murky', 'bright', 'gopro'],
    '2021-12-09_14-53-17_B4': ['clear', 'bright', 'gopro'],
    '2021-11-12_10-25-21_B4': ['murky', 'dim', 'gopro'],
    '2021-11-03_10-49-48_B4': ['murky', 'dim', 'gopro'],
    '2021-09-10_15-38-02_B5': ['murky', 'dark', 'gopro'],
    '2021-09-03_10-57-20_B5': ['clear', 'bright', 'gopro'],
    '2021-09-03_10-40-53_B5': ['clear', 'bright', 'gopro'],
    'G0019499': ['clear', 'bright', 'gopro'],
    'G0019498': ['clear', 'bright', 'gopro'],
    'GP019603': ['clear', 'bright', 'gopro'],
    'GOPR9603': ['clear', 'bright', 'gopro'],
    'GP029602': ['clear', 'bright', 'gopro'],
    'GP019602': ['clear', 'bright', 'gopro'],
    'GOPR9602': ['clear', 'bright', 'gopro'],
    'GP029599': ['clear', 'bright', 'gopro'],
    'GP019599': ['clear', 'bright', 'gopro'],
    'GOPR9599': ['clear', 'bright', 'gopro'],
    "SHD_8f03-1643670176": ["clear", "bright", "snapit"],
    "SHD_8f03-1643673600": ["clear", "bright", "snapit"],
    "SHD_8f03-1643675400": ["clear", "bright", "snapit"],
    "SHD_8f03-1643936400": ["clear", "bright", "snapit"],
    "SHD_8f03-1659571200": ["clear", "dim", "snapit"],
    "SHD_8f03-1659572100": ["clear", "dim", "snapit"],
    "SHD_8f03-1659830400": ["clear", "dim", "snapit"],
    "SHD_8f03-1659831300": ["clear", "dim", "snapit"],
}

In [5]:
framerates = {}
for video_path in video_paths:
    video = cv.VideoCapture(str(video_path))
    if (video.isOpened() == False):
        print(f"Error opening video at path {video_path}")
        continue
    
    framerates[video_path.stem] = int(video.get(cv.CAP_PROP_FPS))

framerates

{'GX020200': 29,
 'GX010200': 29,
 'GX020199': 29,
 'GX010199': 29,
 'F1_20221116': 29,
 'F4_20221116': 29,
 'GX012191': 29,
 'GX012189': 29,
 'GX012190': 29,
 'GX012188': 29,
 'GX020006': 29,
 'GX010006': 29,
 'GX010033': 29,
 'GX010032': 29,
 'GX040026': 29,
 'GX030026': 29,
 'GX020026': 29,
 'GX010027': 29,
 'GX010026': 29,
 'GX030022': 29,
 'GX020022': 29,
 'GX010022': 29,
 '2022-03-15_11-46-18_B5': 29,
 'GH050197': 29,
 '2021-12-09_14-53-17_B4': 59,
 '2021-11-12_10-25-21_B4': 59,
 '2021-11-03_10-49-48_B4': 59,
 '2021-09-10_15-38-02_B5': 59,
 '2021-09-03_10-57-20_B5': 59,
 '2021-09-03_10-40-53_B5': 59,
 'G0019499': 29,
 'G0019498': 29,
 'GP019603': 59,
 'GOPR9603': 59,
 'GP029602': 59,
 'GP019602': 59,
 'GOPR9602': 59,
 'GP029599': 59,
 'GP019599': 59,
 'GOPR9599': 59,
 'SHD_8f03-1643670176': 15,
 'SHD_8f03-1643673600': 15,
 'SHD_8f03-1643675400': 15,
 'SHD_8f03-1643936400': 15,
 'SHD_8f03-1659571200': 15,
 'SHD_8f03-1659572100': 15,
 'SHD_8f03-1659830400': 15,
 'SHD_8f03-165983130

In [6]:
total_num_frames = {}
for video_path in video_paths:
    video = cv.VideoCapture(str(video_path))
    if (video.isOpened() == False):
        print(f"Error opening video at path {video_path}")
        continue
    
    total_num_frames[video_path.stem] = int(video.get(cv.CAP_PROP_FRAME_COUNT))

total_num_frames

{'GX020200': 3060,
 'GX010200': 9720,
 'GX020199': 3666,
 'GX010199': 9600,
 'F1_20221116': 928,
 'F4_20221116': 918,
 'GX012191': 1628,
 'GX012189': 4273,
 'GX012190': 2281,
 'GX012188': 4543,
 'GX020006': 16704,
 'GX010006': 69120,
 'GX010033': 7601,
 'GX010032': 8427,
 'GX040026': 15537,
 'GX030026': 21120,
 'GX020026': 21120,
 'GX010027': 132,
 'GX010026': 21120,
 'GX030022': 15767,
 'GX020022': 21120,
 'GX010022': 21120,
 '2022-03-15_11-46-18_B5': 10530,
 'GH050197': 9570,
 '2021-12-09_14-53-17_B4': 19200,
 '2021-11-12_10-25-21_B4': 19200,
 '2021-11-03_10-49-48_B4': 19200,
 '2021-09-10_15-38-02_B5': 31860,
 '2021-09-03_10-57-20_B5': 25796,
 '2021-09-03_10-40-53_B5': 31860,
 'G0019499': 13437,
 'G0019498': 17984,
 'GP019603': 51038,
 'GOPR9603': 63720,
 'GP029602': 35600,
 'GP019602': 63720,
 'GOPR9602': 63720,
 'GP029599': 18209,
 'GP019599': 63720,
 'GOPR9599': 63720,
 'SHD_8f03-1643670176': 10881,
 'SHD_8f03-1643673600': 13500,
 'SHD_8f03-1643675400': 13501,
 'SHD_8f03-164393640

In [7]:
num_frames_with_obs = {}
for output_dir in base_dir.glob("output/*"):
    frame_paths = output_dir.glob("*/frames/*.jpg")
    frame_paths = sorted(frame_paths)
    num_frames_with_obs[output_dir.name] = len(frame_paths)

num_frames_with_obs

{'GX010026': 3349,
 'F1_20221116': 283,
 'SHD_8f03-1659572100': 964,
 '2022-03-15_11-46-18_B5': 3227,
 'GX010033': 2787,
 'GX020199': 673,
 'SHD_8f03-1643670176': 170,
 'SHD_8f03-1659831300': 559,
 'GX030022': 1744,
 'GP019599': 3342,
 'SHD_8f03-1659830400': 572,
 'GOPR9602': 5568,
 'GX010199': 1794,
 'GOPR9599': 591,
 'GX012191': 130,
 'F4_20221116': 263,
 'GX030026': 3772,
 'GH050197': 1794,
 'GP029602': 4343,
 'GOPR9603': 10667,
 'GX010032': 3092,
 'GX012190': 224,
 'GX020022': 2622,
 'G0019499': 3748,
 '2021-09-03_10-57-20_B5': 6592,
 '2021-12-09_14-53-17_B4': 5246,
 '2021-09-03_10-40-53_B5': 11115,
 'SHD_8f03-1643936400': 44,
 'GX020006': 23,
 '2021-11-03_10-49-48_B4': 6753,
 'GX012188': 1013,
 'GP029599': 1022,
 'GX010006': 1074,
 '2021-09-10_15-38-02_B5': 5994,
 'GP019603': 9455,
 'SHD_8f03-1643675400': 94,
 'GX010200': 1155,
 'G0019498': 5319,
 'GX012189': 221,
 'GX040026': 3091,
 'GX010022': 2345,
 'GX020200': 193,
 'SHD_8f03-1659571200': 484,
 'GP019602': 9017,
 'GX020026': 3

In [8]:
num_individuals = {}
for output_dir in base_dir.glob("output/*"):
    individual_paths = output_dir.glob("*/individuals/*.jpg")
    individual_paths = sorted(individual_paths)
    num_individuals[output_dir.name] = len(individual_paths) 

num_individuals

{'GX010026': 3803,
 'F1_20221116': 388,
 'SHD_8f03-1659572100': 1032,
 '2022-03-15_11-46-18_B5': 4152,
 'GX010033': 5076,
 'GX020199': 819,
 'SHD_8f03-1643670176': 179,
 'SHD_8f03-1659831300': 582,
 'GX030022': 1927,
 'GP019599': 3373,
 'SHD_8f03-1659830400': 589,
 'GOPR9602': 5696,
 'GX010199': 2187,
 'GOPR9599': 591,
 'GX012191': 152,
 'F4_20221116': 366,
 'GX030026': 4254,
 'GH050197': 2125,
 'GP029602': 4591,
 'GOPR9603': 11723,
 'GX010032': 5137,
 'GX012190': 260,
 'GX020022': 2895,
 'G0019499': 4712,
 '2021-09-03_10-57-20_B5': 15086,
 '2021-12-09_14-53-17_B4': 7037,
 '2021-09-03_10-40-53_B5': 26033,
 'SHD_8f03-1643936400': 56,
 'GX020006': 23,
 '2021-11-03_10-49-48_B4': 9454,
 'GX012188': 1362,
 'GP029599': 1034,
 'GX010006': 1083,
 '2021-09-10_15-38-02_B5': 6915,
 'GP019603': 10745,
 'SHD_8f03-1643675400': 96,
 'GX010200': 1309,
 'G0019498': 6920,
 'GX012189': 235,
 'GX040026': 3568,
 'GX010022': 2598,
 'GX020200': 207,
 'SHD_8f03-1659571200': 496,
 'GP019602': 9662,
 'GX020026'

In [9]:
def get_median_gap(frame_paths, min_gap_size=10): 
    gap_list = [] #new gap list for new video 
    
    for index, _ in enumerate(frame_paths[:-1]): 
        # get frame number from file  name 
        curr_frame_path = frame_paths[index]
        next_frame_path = frame_paths[index+1]
        
        curr_frame_num = int(curr_frame_path.stem.split("_")[-1])
        next_frame_num = int(next_frame_path.stem.split("_")[-1])
        gap = next_frame_num - curr_frame_num

        if gap > min_gap_size: 
            gap_list.append(gap) 
    
    if not gap_list: #check if list is empty  
        return None
    else:
        return np.median(gap_list)

# Add median number of frames between successive observations
median_frames_between_obs = {}

# loop though all videos:
for output_dir in base_dir.glob("output/*"): #output/* "output/G0019498"
    frame_paths = output_dir.glob("*/frames/*.jpg")
    frame_paths = sorted(frame_paths)
    median_frames_between_obs[output_dir.name] = get_median_gap(frame_paths) 
    
median_frames_between_obs

{'GX010026': 34.0,
 'F1_20221116': 14.0,
 'SHD_8f03-1659572100': 72.0,
 '2022-03-15_11-46-18_B5': 18.0,
 'GX010033': 18.0,
 'GX020199': 18.0,
 'SHD_8f03-1643670176': 49.0,
 'SHD_8f03-1659831300': 87.0,
 'GX030022': 40.0,
 'GP019599': 134.0,
 'SHD_8f03-1659830400': 110.0,
 'GOPR9602': 106.0,
 'GX010199': 18.0,
 'GOPR9599': 234.0,
 'GX012191': 28.0,
 'F4_20221116': 16.0,
 'GX030026': 34.0,
 'GH050197': 26.0,
 'GP029602': 56.0,
 'GOPR9603': 52.0,
 'GX010032': 14.0,
 'GX012190': 24.0,
 'GX020022': 38.0,
 'G0019499': 22.0,
 '2021-09-03_10-57-20_B5': 16.0,
 '2021-12-09_14-53-17_B4': 24.0,
 '2021-09-03_10-40-53_B5': 18.0,
 'SHD_8f03-1643936400': 22.0,
 'GX020006': 1040.0,
 '2021-11-03_10-49-48_B4': 20.0,
 'GX012188': 17.0,
 'GP029599': 138.0,
 'GX010006': 127.0,
 '2021-09-10_15-38-02_B5': 28.0,
 'GP019603': 54.0,
 'SHD_8f03-1643675400': 74.0,
 'GX010200': 22.0,
 'G0019498': 20.0,
 'GX012189': 43.0,
 'GX040026': 28.0,
 'GX010022': 32.0,
 'GX020200': 32.0,
 'SHD_8f03-1659571200': 74.0,
 'GP0196

In [10]:
rows = []
for video_path in video_paths:
    rows.append(
        (
            video_path,
            framerates[video_path.stem],
            # video_path.parts[5],  # Extract year from path
            total_num_frames[video_path.stem],
            num_frames_with_obs[video_path.stem],
            num_individuals[video_path.stem],  # Add total number of observations (will be higher than total number of frames with observations)
            median_frames_between_obs[video_path.stem],  # Add median number of frames between successive observations
            *video_qualities[video_path.stem]
        )
    )

videos_df = pd.DataFrame(
    rows,
    columns=["FileName", "FrameRate", "TotalNumFrames", "NumFramesWithIndividuals", "NumOfIndividuals", "MedianGap", "WaterQuality", "Brightness", "Camera" ],
)

videos_df["ObservationRatePer1000"] = videos_df["NumFramesWithIndividuals"] / videos_df["TotalNumFrames"] * 1000

videos_df.to_csv("videos_df.csv", index=False)

videos_df

,FileName,FrameRate,TotalNumFrames,NumFramesWithIndividuals,NumOfIndividuals,MedianGap,WaterQuality,Brightness,Camera,ObservationRatePer1000
0,/input/seafood/VIDEOS/snapper/2022/2022-12-20_...,29,3060,193,207,32.0,clear,bright,gopro,63.071895
1,/input/seafood/VIDEOS/snapper/2022/2022-12-20_...,29,9720,1155,1309,22.0,clear,bright,gopro,118.827160
2,/input/seafood/VIDEOS/snapper/2022/2022-12-20_...,29,3666,673,819,18.0,clear,bright,gopro,183.578833
3,/input/seafood/VIDEOS/snapper/2022/2022-12-20_...,29,9600,1794,2187,18.0,clear,bright,gopro,186.875000
4,/input/seafood/VIDEOS/snapper/2022/2022-11-16_...,29,928,283,388,14.0,murky,dim,phone,304.956897
5,/input/seafood/VIDEOS/snapper/2022/2022-11-16_...,29,918,263,366,16.0,murky,dim,phone,286.492375
6,/input/seafood/VIDEOS/snapper/2022/2022-11-15_...,29,1628,130,152,28.0,murky,bright,gopro,79.852580
7,/input/seafood/VIDEOS/snapper/2022/2022-11-15_...,29,4273,221,235,43.0,murky,bright,gopro,51.720103
8,/input/seafood/VIDEOS/snapper/2022/2022-11-15_...,29,2281,224,260,24.0,murky,bright,gopro,98.202543
9,/input/seafood/VIDEOS/snapper/2022/2022-11-15_...,29,4543,1013,1362,17.0,murky,bright,gopro,222.980409


## Find number of spots per observation:
median number of spots per video 

In [20]:
spot_counts = []

for output_dir in tqdm(sorted(base_dir.glob("output/*"))):  # edit back to * for all videos 

    spot_paths = output_dir.glob("*/spots/*.json") #for each json file in spots folder
    spot_paths = sorted(spot_paths)
    
    # To show violin plots spot count distribution against water quality etc,
    # we want a dataframe with:
    # video name, individual ID, water quality, lighting, device, spot count
    
    for spot_path in tqdm(spot_paths):
        image = cv.imread(str(spot_path.with_suffix("")))
        if image is None:
            continue

        observation_area = image.shape[0] * image.shape[1]
        
        with open(spot_path) as f:
            spot_predictions = json.load(f)
        
        # Extract the first frame number and the individual ID from the filename
        first_frame_num = int(spot_path.name.split("_")[-2])
        individual_id = spot_path.name.split("_")[-1].removesuffix(".jpg.json")
        
        # Update the ID to include the video chunk number (otherwise they aren't unique)
        video_chunk_num = int(first_frame_num / 1000)
        individual_id = f"{video_chunk_num}_{individual_id}"
        
        # Add the data as a row for the dataframe
        spot_counts.append([
            spot_path,
            first_frame_num,
            individual_id,
            *video_qualities[output_dir.name],
            observation_area,
            len(spot_predictions)
        ])

observations_df = pd.DataFrame(
    spot_counts, 
    columns=["ObservationPath", "FirstFrameNumber", "IndividualID", "WaterQuality", "Brightness", "Camera", "ObservationAreaPx", "NumSpots"]
)

observations_df.to_csv("observations_df.csv", index=False)

observations_df

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/26033 [00:00<?, ?it/s]

  0%|          | 0/15086 [00:00<?, ?it/s]

  0%|          | 0/6915 [00:00<?, ?it/s]

  0%|          | 0/9454 [00:00<?, ?it/s]

...

  0%|          | 0/496 [00:00<?, ?it/s]

  0%|          | 0/1032 [00:00<?, ?it/s]

  0%|          | 0/589 [00:00<?, ?it/s]

  0%|          | 0/582 [00:00<?, ?it/s]

,ObservationPath,FirstFrameNumber,IndividualID,WaterQuality,Brightness,Camera,ObservationAreaPx,NumSpots
0,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,926,0_2,clear,bright,gopro,386757,0
1,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,928,0_2,clear,bright,gopro,358092,0
2,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,930,0_2,clear,bright,gopro,405040,0
3,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,930,0_3,clear,bright,gopro,899391,0
4,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,932,0_3,clear,bright,gopro,968841,3
...,...,...,...,...,...,...,...,...
186795,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,13490,13_20113,clear,dim,snapit,167958,0
186796,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,13492,13_20113,clear,dim,snapit,181800,0
186797,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,13494,13_20113,clear,dim,snapit,180846,0
186798,/workspace/hrrsxb/AIP/Morphometrics/workflows/...,13496,13_20113,clear,dim,snapit,204462,0
